# Load and import libraries

In [7]:
pip install --upgrade datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 12.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 

In [1]:
import datasets
from datasets import load_dataset
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments, DataCollatorForLanguageModeling
import torch

In [2]:
datasets.utils.logging.set_verbosity_info()


# Loading datasets for Text summarization and question answering

In [8]:
data = load_dataset("squad")

Overwrite dataset info from restored data version if exists.
INFO:datasets.builder:Overwrite dataset info from restored data version if exists.
Loading Dataset info from /root/.cache/huggingface/datasets/squad/plain_text/0.0.0/7b6d24c440a36b6815f21b70d25016731768db1f
INFO:datasets.info:Loading Dataset info from /root/.cache/huggingface/datasets/squad/plain_text/0.0.0/7b6d24c440a36b6815f21b70d25016731768db1f
Found cached dataset squad (/root/.cache/huggingface/datasets/squad/plain_text/0.0.0/7b6d24c440a36b6815f21b70d25016731768db1f)
INFO:datasets.builder:Found cached dataset squad (/root/.cache/huggingface/datasets/squad/plain_text/0.0.0/7b6d24c440a36b6815f21b70d25016731768db1f)
Loading Dataset info from /root/.cache/huggingface/datasets/squad/plain_text/0.0.0/7b6d24c440a36b6815f21b70d25016731768db1f
INFO:datasets.info:Loading Dataset info from /root/.cache/huggingface/datasets/squad/plain_text/0.0.0/7b6d24c440a36b6815f21b70d25016731768db1f


In [4]:
dataset = load_dataset("xsum", cache_dir=None)

README.md:   0%|          | 0.00/6.24k [00:00<?, ?B/s]

xsum.py:   0%|          | 0.00/5.76k [00:00<?, ?B/s]

Loading Dataset Infos from /root/.cache/huggingface/modules/datasets_modules/datasets/xsum/082863bf4754ee058a5b6f6525d0cb2b18eadb62c7b370b095d1364050a52b71
INFO:datasets.info:Loading Dataset Infos from /root/.cache/huggingface/modules/datasets_modules/datasets/xsum/082863bf4754ee058a5b6f6525d0cb2b18eadb62c7b370b095d1364050a52b71
Overwrite dataset info from restored data version if exists.
INFO:datasets.builder:Overwrite dataset info from restored data version if exists.
Loading Dataset info from /root/.cache/huggingface/datasets/xsum/default/1.2.0/082863bf4754ee058a5b6f6525d0cb2b18eadb62c7b370b095d1364050a52b71
INFO:datasets.info:Loading Dataset info from /root/.cache/huggingface/datasets/xsum/default/1.2.0/082863bf4754ee058a5b6f6525d0cb2b18eadb62c7b370b095d1364050a52b71
Found cached dataset xsum (/root/.cache/huggingface/datasets/xsum/default/1.2.0/082863bf4754ee058a5b6f6525d0cb2b18eadb62c7b370b095d1364050a52b71)
INFO:datasets.builder:Found cached dataset xsum (/root/.cache/huggingfac

# Exploring the dataset

In [7]:
sample = dataset["train"][0]
print("Sample Article:")
print(sample["document"])
print("\nSample Summary (Highlights):")
print(sample["summary"])

Sample Article:
The full cost of damage in Newton Stewart, one of the areas worst affected, is still being assessed.
Repair work is ongoing in Hawick and many roads in Peeblesshire remain badly affected by standing water.
Trains on the west coast mainline face disruption due to damage at the Lamington Viaduct.
Many businesses and householders were affected by flooding in Newton Stewart after the River Cree overflowed into the town.
First Minister Nicola Sturgeon visited the area to inspect the damage.
The waters breached a retaining wall, flooding many commercial properties on Victoria Street - the main shopping thoroughfare.
Jeanette Tate, who owns the Cinnamon Cafe which was badly affected, said she could not fault the multi-agency response once the flood hit.
However, she said more preventative work could have been carried out to ensure the retaining wall did not fail.
"It is difficult but I do think there is so much publicity for Dumfries and the Nith - and I totally appreciate tha

In [9]:
sample1 = data["train"][0]
print("Sample Question:")
print(sample1["question"])
print("\nSample Answer:")
print(sample1["answers"])
print("\nSample Context:")
print(sample1["context"])

Sample Question:
To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?

Sample Answer:
{'text': ['Saint Bernadette Soubirous'], 'answer_start': [515]}

Sample Context:
Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.


# Samples to work on

In [10]:
small_train_dataset = dataset["train"].select(range(100))
small_eval_dataset = dataset["validation"].select(range(20))

In [11]:
small_train_data = data["train"].select(range(100))
small_eval_data = data["validation"].select(range(20))

# Load the tokenizer and the model

In [12]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [13]:
#GPT-2 does not have a padding token by default, so we set it to the EOS token.
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.pad_token_id

# Preprocess the dataset

In [18]:
def preprocess_function(examples):
    inputs = [document for document in examples["document"]]
    targets = [summary for summary in examples["summary"]]

    model_inputs = tokenizer(inputs, max_length=256, truncation=True, padding="max_length")
    labels = tokenizer(targets, max_length=64, truncation=True, padding="max_length")

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [25]:
def preprocess_func(example):
    inputs = [str(question) for question in example["question"]]
    targets = [str(answers) for answers in example["answers"]]

    model_inputs = tokenizer(inputs, max_length=256, truncation=True, padding="max_length")
    labels = tokenizer(targets, max_length=64, truncation=True, padding="max_length")

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Applying preprocessing to the small training and evaluation datasets

In [19]:
tokenized_train = small_train_dataset.map(preprocess_function, batched=True, remove_columns=small_train_dataset.column_names)
tokenized_eval = small_eval_dataset.map(preprocess_function, batched=True, remove_columns=small_eval_dataset.column_names)

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Caching processed dataset at /root/.cache/huggingface/datasets/xsum/default/1.2.0/082863bf4754ee058a5b6f6525d0cb2b18eadb62c7b370b095d1364050a52b71/cache-8678fdc4fb0abcb0.arrow
INFO:datasets.arrow_dataset:Caching processed dataset at /root/.cache/huggingface/datasets/xsum/default/1.2.0/082863bf4754ee058a5b6f6525d0cb2b18eadb62c7b370b095d1364050a52b71/cache-8678fdc4fb0abcb0.arrow


Map:   0%|          | 0/20 [00:00<?, ? examples/s]

Caching processed dataset at /root/.cache/huggingface/datasets/xsum/default/1.2.0/082863bf4754ee058a5b6f6525d0cb2b18eadb62c7b370b095d1364050a52b71/cache-2a58349d8ee2327d.arrow
INFO:datasets.arrow_dataset:Caching processed dataset at /root/.cache/huggingface/datasets/xsum/default/1.2.0/082863bf4754ee058a5b6f6525d0cb2b18eadb62c7b370b095d1364050a52b71/cache-2a58349d8ee2327d.arrow


In [26]:
tokenized_train1 = small_train_data.map(preprocess_func, batched=True, remove_columns=small_train_data.column_names)
tokenized_eval1 = small_eval_data.map(preprocess_func, batched=True, remove_columns=small_eval_data.column_names)

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Caching processed dataset at /root/.cache/huggingface/datasets/squad/plain_text/0.0.0/7b6d24c440a36b6815f21b70d25016731768db1f/cache-9b7f50633e5cdab4.arrow
INFO:datasets.arrow_dataset:Caching processed dataset at /root/.cache/huggingface/datasets/squad/plain_text/0.0.0/7b6d24c440a36b6815f21b70d25016731768db1f/cache-9b7f50633e5cdab4.arrow


Map:   0%|          | 0/20 [00:00<?, ? examples/s]

Caching processed dataset at /root/.cache/huggingface/datasets/squad/plain_text/0.0.0/7b6d24c440a36b6815f21b70d25016731768db1f/cache-96459fa14abbbb9e.arrow
INFO:datasets.arrow_dataset:Caching processed dataset at /root/.cache/huggingface/datasets/squad/plain_text/0.0.0/7b6d24c440a36b6815f21b70d25016731768db1f/cache-96459fa14abbbb9e.arrow


In [27]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# Setting the trainer

In [28]:
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=1,
    weight_decay=0.01,
    save_total_limit=1,
    logging_dir="./logs",
    logging_steps=10,
    report_to="none",
)

In [29]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_eval,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

<ipython-input-29-21fb1384e29e>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [30]:
trainer1 = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train1,
    eval_dataset=tokenized_eval1,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

<ipython-input-30-ecccee96e414>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer1 = Trainer(


# Finetune the model

In [31]:
trainer.train()

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Epoch,Training Loss,Validation Loss
1,3.116300,3.054092


TrainOutput(global_step=50, training_loss=3.208329772949219, metrics={'train_runtime': 535.763, 'train_samples_per_second': 0.187, 'train_steps_per_second': 0.093, 'total_flos': 13064601600000.0, 'train_loss': 3.208329772949219, 'epoch': 1.0})

In [32]:
trainer1.train()

Epoch,Training Loss,Validation Loss
1,3.648500,3.532412


TrainOutput(global_step=50, training_loss=3.6513763427734376, metrics={'train_runtime': 549.2934, 'train_samples_per_second': 0.182, 'train_steps_per_second': 0.091, 'total_flos': 13064601600000.0, 'train_loss': 3.6513763427734376, 'epoch': 1.0})

# Save the Fine-Tuned Model

In [33]:
trainer.save_model("./fine_tuned_gpt2_demo1")
tokenizer.save_pretrained("./fine_tuned_gpt2_demo1")

('./fine_tuned_gpt2_demo1/tokenizer_config.json',
 './fine_tuned_gpt2_demo1/special_tokens_map.json',
 './fine_tuned_gpt2_demo1/vocab.json',
 './fine_tuned_gpt2_demo1/merges.txt',
 './fine_tuned_gpt2_demo1/added_tokens.json')

In [34]:
trainer1.save_model("./fine_tuned_gpt2_demo2")
tokenizer.save_pretrained("./fine_tuned_gpt2_demo2")

('./fine_tuned_gpt2_demo2/tokenizer_config.json',
 './fine_tuned_gpt2_demo2/special_tokens_map.json',
 './fine_tuned_gpt2_demo2/vocab.json',
 './fine_tuned_gpt2_demo2/merges.txt',
 './fine_tuned_gpt2_demo2/added_tokens.json')

# Generate summary with my finetuned model

In [35]:
input_text = """
NASA’s Perseverance rover has successfully landed on Mars and has begun its mission to search for signs of ancient microbial life.
This marks another milestone in humanity’s exploration of the Red Planet.
"""

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = model.to(device)

inputs = tokenizer(
    input_text,
    return_tensors="pt",
    max_length=256,
    truncation=True,
    padding=True
).to(device)

input_ids = inputs["input_ids"]
attention_mask = inputs["attention_mask"]


model.eval()


outputs = model.generate(
    input_ids=input_ids,
    attention_mask=attention_mask,
    max_length=50,
    min_length=10,
    length_penalty=2.0,
    num_beams=4,
    early_stopping=True
)

summary = tokenizer.decode(outputs[0], skip_special_tokens=True)

print("Generated Summary:", summary)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated Summary: 
NASA’s Perseverance rover has successfully landed on Mars and has begun its mission to search for signs of ancient microbial life.
This marks another milestone in humanity’s exploration of the Red Planet.
NASA's Mars Science Laboratory
